In [ ]:
import tensorflow as tf
import numpy as np
import requests
from PIL import Image
from io import BytesIO

# Load Fashion MNIST dataset
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Preprocess the data
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255

# Using ImageDataGenerator for data augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Define the model architecture with BatchNormalization and Dropout
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Fit the model on the augmented data generator
model.fit(datagen.flow(train_images, train_labels, batch_size=64),
          epochs=1,
          validation_data=(test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255, test_labels))

model.save('fashion_mnist_model.h5')

# Load the trained model
model.load_weights('fashion_mnist_model.h5')

# Class labels for Fashion MNIST dataset
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Function to classify input image
def classify_clothing(image_url):
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content)).convert('L')
    img_resized = img.resize((28, 28))
    img_array = np.array(img_resized) / 255.0
    img_input = img_array.reshape((1, 28, 28, 1))
    prediction = model.predict(img_input)
    predicted_class = np.argmax(prediction)
    return class_names[predicted_class]


In [ ]:
link = input("Enter the clothing image link here: ")
print(classify_clothing(link))